### 提取文本_表格文件里的图片

In [1]:
import fitz  # PyMuPDF

pdf = fitz.open(r"D:\desktop\宏利 pdf 文件\文本_表格\电子行政运作手册_保单行政_2023_10.pdf")
page = pdf.load_page(5)
images = page.get_images()  # 获取页面所有图像
for img_index, img in enumerate(images):
    xref = img[0]
    base_image = pdf.extract_image(xref)
    with open(f"D:\desktop\宏利 pdf 文件\文本_表格\chart_{5}_{img_index}.png", "wb") as f:
        f.write(base_image["image"])  # 保存为PNG

In [ ]:
##图1
from paddleocr import PPStructure, save_structure_res
import cv2
import os
import pandas as pd
table_engine = PPStructure(layout=True, show_log=True)

img_path = r"D:\desktop\code\chart_5_2.png"
img = cv2.imread(img_path)

result = table_engine(img)

# for line in result:
#     print(line)

table_data = []
for line in result:
    if 'res' in line:
        for cell in line['res']:
            table_data.append([cell['text']])

df = pd.DataFrame(table_data)
df.to_excel("D:\desktop\宏利 pdf 文件\文本_表格\chart1.xlsx", index=False)

In [1]:
import torch

x = torch.tensor([1.0, 2.0])
print(x)


tensor([1., 2.])


In [1]:
import paddle
paddle.utils.run_check()


d:\naconda\envs\ocr\lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 CPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [3]:
from paddleocr import PPStructure, save_structure_res
import cv2
import os
import pandas as pd
import re
table_engine = PPStructure(layout=True, show_log=True)

img_path = r"D:\desktop\code\chart_5_3.png"
img = cv2.imread(img_path)

result = table_engine(img)

# for line in result:
#     print(line)
table_data = []
# 使用正则表达式从HTML中提取文本
for line in result:
    if 'res' in line and isinstance(line['res'], dict):
        if 'html' in line['res']:
            html_content = line['res']['html']
            # 提取所有<td>或<th>标签中的内容
            cell_texts = re.findall(r'<t[dh][^>]*>(.*?)</t[dh]>', html_content)
            for cell_text in cell_texts:
                if cell_text.strip():
                    table_data.append([cell_text.strip()])

df = pd.DataFrame(table_data)
df.to_excel(r"D:\desktop\宏利 pdf 文件\文本_表格\chart1.xlsx", index=False)

[2025/04/09 22:13:51] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\36325/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\36325/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320',

### 提取彩页

In [18]:
import fitz  # PyMuPDF

def extract_text_from_color_pages(input_pdf, output_txt):
    doc = fitz.open(input_pdf)
    with open(output_txt, "w", encoding="utf-8") as f:
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap()
            if pix.colorspace.n != 1:  # 检查是否为非灰度（彩色）
                text = page.get_text()
                if text.strip():  # 确保提取的文本非空
                    f.write(f"Page {page_num + 1}:\n{text}\n\n")

# 调用函数
extract_text_from_color_pages(
    r"D:\desktop\宏利 pdf 文件\彩页\宏摯傳承保障計劃_产品彩页.pdf",
    r"D:\desktop\宏利 pdf 文件\彩页\宏摯傳承保障計劃_产品彩页.txt"
)

In [20]:
# 使用 PyMuPDF 提取并按段落整理
# 可以通过 PyMuPDF 的 get_text("blocks") 方法提取文本块，并按顺序整理。
import fitz  # PyMuPDF
from zhconv import convert
def extract_and_clean_text(input_pdf, output_txt):
    doc = fitz.open(input_pdf)
    with open(output_txt, "w", encoding="utf-8") as f:
        for page in doc:
            page_dict = page.get_text("dict")
            lines_all = []
            for block in page_dict["blocks"]:
                for line in block.get("lines", []):
                    line_text = "".join(span["text"] for span in line["spans"]).strip()
                    line_text = convert(line_text, 'zh-hans')
                    if line_text:
                        lines_all.append((line["bbox"][1], line_text))  # 用 Y 坐标排序

            lines_all.sort(key=lambda x: x[0])  # 按 Y 排序
            for _, line_text in lines_all:
                f.write(line_text + "\n")
            f.write("\n")

# 调用函数
extract_and_clean_text(
    r"D:\desktop\宏利 pdf 文件\彩页\宏摯傳承保障計劃_产品彩页.pdf",
    r"D:\desktop\宏利 pdf 文件\彩页\宏摯傳承保障計劃_产品彩页2.txt"
)

In [22]:
import tabula
tabula.convert_into(r"D:\desktop\宏利 pdf 文件\彩页\宏摯傳承保障計劃_产品彩页.pdf", "./output/彩页.json", output_format="json", pages='all')

[2025-04-09 11:52:47,153] [ WARNING] backend.py:56 - Failed to import jpype dependencies. Fallback to subprocess.
[2025-04-09 11:52:47,168] [ WARNING] backend.py:59 - No module named 'jpype'


In [23]:
import tabula
tabula.convert_into(r"D:\desktop\宏利 pdf 文件\彩页\宏摯傳承保障計劃_产品彩页.pdf", "./output/彩页.csv", output_format="csv", pages='all')